In [32]:
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.python.keras.layers import Dense, Flatten
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
import numpy as np
import tensorflow_datasets as tfds
import tensorflow as tf
import matplotlib.pyplot as plt;
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications import Xception
from tensorflow.keras.applications.xception import preprocess_input, decode_predictions
from tensorflow.keras.preprocessing import image
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import Sequential
import os as os
import cv2

In [33]:
import numpy as np
import numpy.random as npr
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('bmh')

# Preparing The Data

### load you data here

In [34]:
#data:
data = np.load('Copy of data_hard_test.npy')
#labels:
labelss = np.load('Copy of labels_hard_test.npy')


In [35]:
labelss.shape

(2544, 1)

In [36]:
nn=data.shape[1]

In [37]:
n=220
data = data.T;
data = data.reshape(nn,300,300,3)

In [38]:
data = tf.keras.layers.Resizing(n, n)(data)

In [39]:
labels = to_categorical(labelss,num_classes=9)

In [40]:
data = np.array(data)

In [41]:
data=data/255

In [42]:
data = tf.data.Dataset.from_tensor_slices((data, labels))

# Loading the weights


In [43]:
from tensorflow.keras.layers import Input, Flatten, Dense
from tensorflow.keras.models import Model
import tensorflow as tf
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.layers import Dropout
 
lr_scheduler = ReduceLROnPlateau(
    monitor='val_loss',  # You can also use 'val_accuracy' or other metrics
    factor=0.2,           # Factor by which the learning rate will be reduced
    patience=6,          # Number of epochs with no improvement after which learning rate will be reduced
    min_lr=1e-6           # Lower bound on the learning rate
)

# Create the Xception base model
base_model = tf.keras.applications.Xception(
    include_top=False,
    weights="imagenet",
    input_shape=(n, n, 3),
    pooling='avg'
)
base_model.trainable = False
x = Flatten()(base_model.output)
base_model.trainable = True
# Fine-tune only the last few layers
fine_tune_at = 100
for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False

#base_model.trainable = True

# Fine-tune only the last few layers

x = Dense(300, activation='relu')(x)#, kernel_regularizer=tf.keras.regularizers.l2(0.001))(x)
x=Dropout(0.60)(x)
#x = Dense(100, activation='relu')(x)#, kernel_regularizer=tf.keras.regularizers.l2(0.001))(x)
#x=Dropout(0.52)(x)
x = Dense(27, activation='relu')(x)
x=Dropout(0.5)(x)
outputs = Dense(9, activation='softmax')(x)

# Create the full model
xception_model = Model(inputs=base_model.input, outputs=outputs)
xception_model.load_weights('best_weights7.h5')

In [44]:
from tensorflow.keras.optimizers import Nadam
optimizer = Nadam(learning_rate=0.0001, beta_1=0.96, beta_2=0.998)
xception_model.compile(optimizer=optimizer,loss='categorical_crossentropy',metrics=['accuracy'])
batch_size=73

# Making Predictions

## evaluating the effect of threshold

In [45]:
threshold = 0.8  # Define your threshold value

# Evaluate the model while applying the threshold
total_samples = 0
correct_samples = 0

for batch in data.batch(batch_size):
    images, labels = batch
    predictions = xception_model.predict(images)

    for i in range(len(predictions)):
        predicted_class = np.argmax(predictions[i])
        max_probability = np.max(predictions[i])

        if max_probability >= threshold:
            total_samples += 1
            if predicted_class == np.argmax(labels[i]):
                correct_samples += 1
        

accuracy_with_threshold = correct_samples / total_samples if total_samples > 0 else 0
accuracy_with_threshold_alldata = correct_samples / nn 

print(f"Accuracy with threshold {threshold} mislabeled excluded : {accuracy_with_threshold}")
print(f"Accuracy with threshold {threshold} mislabeled included : {accuracy_with_threshold_alldata}")

2/2 [==============================] - 1s 567ms/step
Accuracy with threshold 0.8 mislabeled excluded : 0.8042553191489362
Accuracy with threshold 0.8 mislabeled included : 0.5200471698113207


## Updating predicted labels (unrecognized data are labeled as -1)

In [46]:
predictions_with_labels=[]
for batch in data.batch(batch_size):
    images, labels = batch
    predictions = xception_model.predict(images)

    for i in range(len(predictions)):
        predicted_class = np.argmax(predictions[i])
        max_probability = np.max(predictions[i])

        if max_probability >= threshold:
            predictions_with_labels.append(predicted_class)
        else:
            # Assign 'mislabeled' label to uncertain predictions
            predictions_with_labels.append(-1)

# Now you have a list 'predictions_with_labels' containing all predicted labels, including 'mislabeled' for uncertain predictions

2/2 [==============================] - 1s 533ms/step


In [47]:
new_labels=predictions_with_labels

In [48]:
predictions_with_labels=to_categorical(labelss,num_classes=10)

In [49]:
new_labels = np.array(new_labels)

In [50]:
np.where(new_labels==-1)

(array([  31,   32,   34,   37,   43,   44,   95,   98,  112,  165,  170,
         172,  174,  176,  177,  178,  179,  225,  240,  244,  245,  246,
         247,  251,  254,  285,  286,  287,  288,  289,  290,  291,  292,
         293,  294,  295,  296,  297,  298,  299,  300,  301,  302,  303,
         304,  305,  307,  308,  309,  310,  311,  312,  313,  314,  329,
         330,  331,  332,  333,  334,  336,  338,  339,  340,  342,  343,
         344,  361,  364,  369,  371,  385,  389,  393,  395,  402,  404,
         435,  438,  439,  442,  444,  447,  456,  464,  496,  499,  500,
         504,  506,  507,  508,  513,  515,  520,  523,  570,  573,  575,
         576,  578,  580,  637,  640,  645,  646,  648,  649,  651,  653,
         658,  693,  700,  704,  707,  708,  713,  715,  719,  765,  766,
         767,  769,  770,  771,  772,  778,  779,  780,  785,  786,  790,
         800,  829,  832,  833,  835,  836,  838,  840,  863,  864,  869,
         872,  874,  875,  879,  880, 

## Final accuracy: the updated predicted labels compared with ground truth

In [51]:
correct_sampless=0
for i in range(len(new_labels)):
        if new_labels[i] == labelss[i]:
            correct_sampless += 1
accuracy_with_threshold_alldata1 = correct_sampless / nn 
print(f"Accuracy with threshold {threshold} mislabeled included : {accuracy_with_threshold_alldata1}")

Accuracy with threshold 0.8 mislabeled included : 0.7653301886792453
